In [ ]:

month = ['here.ta_201901','here.ta_201902', 'here.ta_201903', 'here.ta_201904', 'here.ta_201905', 'here.ta_201906', 'here.ta_201907', 'here.ta_201908', 'here.ta_201909', 'here.ta_201910', 'here.ta_201911', 'here.ta_201912' ]
for i in month:
    print(i)
    with con:
        with con.cursor() as cur:
            sql = '''INSERT INTO congestion.speeds_links_30_v5_append_tobedeleted

                    with here_prep as (
                    select link_dir, length, tx, stddev, confidence, mean, pct_50, mean_corr, pct_50_corr from 
                    {0}
                    inner join congestion.segment_links_v5 using (link_dir))
                    , here as (
                    select * from here_prep
                    LEFT JOIN ref.holiday hol ON hol.dt = here_prep.tx::date
                    where hol.dt IS NULL AND date_part('isodow'::text, tx::date)::integer < 6)	

                    SELECT X.segment_id, X.link_dir, X.length AS link_length, X.datetime_bin, 
                    X.spd_avg_all, X.spd_avg_all_corr, Y.spd_avg_hc, Y.spd_avg_hc_corr, 
                    X.spd_med_all, X.spd_med_all_corr, Y.spd_med_hc, Y.spd_med_hc_corr,
                    X.count_all, Y.count_hc
                    FROM
                    (
                    SELECT a.segment_id, a.link_dir, 
                    (datetime_bin(b.tx,30)) AS datetime_bin,
                    harmean(mean) AS spd_avg_all,
                    harmean(pct_50) AS spd_med_all,
                    harmean(mean_corr) AS spd_avg_all_corr,
                    harmean(pct_50_corr) AS spd_med_all_corr, 			  
                    COUNT (DISTINCT b.tx) AS count_all, b.length
                    FROM congestion.segment_links_v5 a
                    INNER JOIN here b
                    USING (link_dir)
                    GROUP BY a.segment_id, a.link_dir, datetime_bin, b.length
                    ) X

                    LEFT JOIN

                    (
                    SELECT a.segment_id, a.link_dir, 
                    (datetime_bin(b.tx,30)) AS datetime_bin,
                    harmean(mean) AS spd_avg_hc,
                    harmean(pct_50) AS spd_med_hc,
                    harmean(mean_corr) AS spd_avg_hc_corr,
                    harmean(pct_50_corr) AS spd_med_hc_corr, 			  
                    COUNT (DISTINCT b.tx)  AS count_hc, b.length
                    FROM congestion.segment_links_v5 a
                    INNER JOIN here b
                    USING (link_dir)
                    WHERE confidence >= 30
                    GROUP BY a.segment_id, a.link_dir, datetime_bin, b.length
                        )  Y

                    USING (segment_id, link_dir, datetime_bin, length)
                    ORDER BY segment_id, link_dir, datetime_bin'''.format(i)
            cur.execute(sql)